In [1]:
def Bloch_simul_rot(self, x,y,z,T1,T2,RF,Gz,time_step,slice_thick,spatial_point,off_range):
        T1 = T1 / 1000

        T2 = T2 / 1000

        length_RF = int(RF[1,:].size)

        t_int = time_step * 10 ** (- 3)

        # delta_omega = 2*pi*42.57747892*10^6 * (-1:2/(spatial_point-1):1).'*slice_thick*0.001*Gz;
        delta_omega = np.pi*off_range.repeat(1,256)
        RF_amp = (RF[1,:] * 2 * np.pi * 4257.747892).repeat(spatial_point, 1)
        RF_phase = (RF[2,:]).repeat(spatial_point, 1)
        alpha = t_int * torch.sqrt(RF_amp ** 2 + delta_omega **2)
        zeta = torch.atan2(RF_amp, delta_omega)

        theta = RF_phase
        ca = torch.cos(alpha)
        sa = torch.sin(alpha)
        cz = torch.cos(zeta)
        sz = torch.sin(zeta)
        ct = torch.cos(theta)
        st = torch.sin(theta)
        E1 = torch.exp(- t_int / T1)
        E2 = torch.exp(- t_int / T2)
        Mx_x_part = torch.matmul(ct,(torch.matmul(torch.matmul(E2,ct),sz ** 2) + torch.matmul(cz,(torch.matmul(torch.matmul(E2,sa),st) + torch.matmul(torch.matmul(torch.matmul(E2,ca),ct),cz))))) + torch.matmul(st,(torch.matmul(torch.matmul(E2,ca),st) - torch.matmul(torch.matmul(torch.matmul(E2,ct),cz),sa)))
        Mx_y_part = torch.matmul(st,(torch.matmul(torch.matmul(E2,ct),sz ** 2) + torch.matmul(cz,(torch.matmul(torch.matmul(E2,sa),st) + torch.matmul(torch.matmul(torch.matmul(E2,ca),ct),cz))))) - torch.matmul(ct,(torch.matmul(torch.matmul(E2,ca),st) - torch.matmul(torch.matmul(torch.matmul(E2,ct),cz),sa)))
        Mx_z_part = torch.matmul(torch.matmul(torch.matmul(E2,ct),cz),sz) - torch.matmul(sz,(torch.matmul(torch.matmul(E2,sa),st) + torch.matmul(torch.matmul(torch.matmul(E2,ca),ct),cz)))
        My_x_part = torch.matmul(- ct,(torch.matmul(torch.matmul(- E2,st),sz ** 2) + torch.matmul(cz,(torch.matmul(torch.matmul(E2,ct),sa) - torch.matmul(torch.matmul(torch.matmul(E2,ca),cz),st))))) - torch.matmul(st,(torch.matmul(torch.matmul(E2,ca),ct) + torch.matmul(torch.matmul(torch.matmul(E2,cz),sa),st)))
        My_y_part = torch.matmul(ct,(torch.matmul(torch.matmul(E2,ca),ct) + torch.matmul(torch.matmul(torch.matmul(E2,cz),sa),st))) - torch.matmul(st,(torch.matmul(torch.matmul(- E2,st),sz ** 2) + torch.matmul(cz,(torch.matmul(torch.matmul(E2,ct),sa) - torch.matmul(torch.matmul(torch.matmul(E2,ca),cz),st)))))
        My_z_part = torch.matmul(sz,(torch.matmul(torch.matmul(E2,ct),sa) - torch.matmul(torch.matmul(torch.matmul(E2,ca),cz),st))) + torch.matmul(torch.matmul(torch.matmul(E2,cz),st),sz)
        Mz_x_part = torch.matmul(ct,(torch.matmul(torch.matmul(E1,cz),sz) - torch.matmul(torch.matmul(torch.matmul(E1,ca),cz),sz))) + torch.matmul(torch.matmul(torch.matmul(E1,sa),st),sz)
        Mz_y_part = torch.matmul(st,(torch.matmul(torch.matmul(E1,cz),sz) - torch.matmul(torch.matmul(torch.matmul(E1,ca),cz),sz))) - torch.matmul(torch.matmul(torch.matmul(E1,ct),sa),sz)
        Mz_z_part = torch.matmul(E1,cz ** 2) + torch.matmul(torch.matmul(E1,ca),sz ** 2)

    
        rot = torch.zeros(spatial_point,length_RF,3,3)
        rot[:,:,1,1] = Mx_x_part
        rot[:,:,1,2] = Mx_y_part
        rot[:,:,1,3] = Mx_z_part
        rot[:,:,2,1] = My_x_part
        rot[:,:,2,2] = My_y_part
        rot[:,:,2,3] = My_z_part
        rot[:,:,3,1] = Mz_x_part
        rot[:,:,3,2] = Mz_y_part
        rot[:,:,3,3] = Mz_z_part
        return rot


In [2]:
    def RF_simul(self, RF_pulse, RF_img, off_range, time_step):
        size = int(RF_pulse.size)
        RF_pulse_new = torch.zeros((size,2))
        RF_pulse_new[:,1] = torch.sqrt(RF_pulse**2+RF_img**2)
        RF_pulse_new[:,2] = torch.angle(torch.complex(RF_pulse, RF_img))
        
        max_rf_amp = torch.max(RF_pulse_new[:,1]) / (2*torch.pi *42.577*1e+6*time_step*1e-4)
        rr = 42.577; # MHz/T
        Gz =  40; # mT/m (fixed)
        pos = torch.abs(off_range[0] / rr / Gz); # mm
        pulse = np.transpose(np.array([RF_pulse_new[:,1] / np.max(RF_pulse_new[:,1]) * max_rf_amp,RF_pulse_new[:,2] / pi * 180]))
        gg = np.transpose(np.ones((RF_pulse_new.shape[1-1],1))) * Gz
        
        freq_shape = int(off_range.size)
        rot = Bloch_simul_rot(np.zeros([freq_shape,1]),np.zeros([freq_shape,1]),np.ones([freq_shape,1]),1e+10,1e+10,pulse,gg,time_step * 1e+3,pos * 0.001,freq_shape,off_range)
        
        return rot

In [4]:
from scipy.io import loadmat

a = loadmat('./SLR_inv_oc.mat')

FileNotFoundError: [Errno 2] No such file or directory: './SLR_inv_oc.mat'

In [46]:
k = torch.tensor(0.2)
torch.angle(torch.complex(k,k))

k =torch.zeros(2,2)
a = k.repeat(1,256)
a = np.pi*k.repeat(1,256)

In [47]:
a.shape

torch.Size([2, 512])

In [50]:
k = torch.ones(612,1)
k = k[0:3:6]